# モデルの学習について
「モデルの学習においてどのようにパラメータを更新するか」ですが、__誤差逆伝播法__をというアルゴリズムでパラメータを更新します。誤差逆伝播法を説明するために、シンプルなフィードフォーワードネットワークでイメージを説明したいと思います。
図においてユニット間の矢印が\は重み$w$（weight）を表し、円の中の文字は各ユニットの名称とします。また、活性化関数はactivationの頭文字をとって$a(\cdot)$で表します。図において、$o_1$はユニット$u_1, u_2$の重み付き和で$o_1 = w_{11}u_1 + w_{21}u_2$で計算できます。
同様に、$u_1 = w_{11}x_1 + w_{21}x_2 + w_{31}u_3 + w_{41}u_4 = \sum_{i=1}^4 w_{i1}x_i  = \mathbf{W}\boldsymbol{x}$とまとめられます。
よって、$\boldsymbol{y} = a(o_1) = a(a(\mathbf{W}_2\boldsymbol{u})) = a(a(\mathbf{W}_2a(\mathbf{W}_1\boldsymbol{x}))$とreurssiveに表せます。


出力を$\boldsymbol{y}$、正解を$\hat{\boldsymbol{y}}$とすると、誤差は誤差関数
$L(\hat{\boldsymbol{y}}, \boldsymbol{y})$で計算されます。
誤差逆伝播法は$\dfrac{\partial L(\hat{\boldsymbol{y}}, \boldsymbol{y})}{\partial w_{ij}} = (\hat{\boldsymbol{y}} -  \boldsymbol{y})^{\top} \dfrac{\partial \hat{\boldsymbol{y}}}{\partial w_{ij}}$をベースに重みを更新します。
$l$層目の更新は$l+1$層目の値をもとにchain ruleを用いて計算します。

微分と聞いて萎えた人もいるかもしれませんが、ご安心ください。フレームワークが自動で計算してくれます。
では、早速、フレームワークを使ってライブラリでの変数の使い方、自動微分を確認しましょう。

各セルはshift + enterで実行できます。

```
import torch
x = torch.Tensor(5, 3)
```
を下のセルで実行してみましょう。

また、`%%bash`を冒頭に書いたセルではbashコマンドを実行できます。

In [15]:
%%bash
wget http://pytorch.org/tutorials/_downloads/tensor_tutorial.ipynb
wget http://pytorch.org/tutorials/_downloads/autograd_tutorial.ipynb

ls -g -h

total 152
-rw-r--r--  1 staff   9.1K May  6 17:53 0_introduction_of _learning.ipynb
-rw-r--r--@ 1 staff   6.3K Apr 23 17:16 autograd_tutorial.ipynb
-rw-r--r--@ 1 staff   6.3K Apr 23 17:16 autograd_tutorial.ipynb.1
-rw-r--r--@ 1 staff   6.3K Apr 23 17:16 autograd_tutorial.ipynb.2
-rw-r--r--@ 1 staff   6.3K Apr 23 17:16 autograd_tutorial.ipynb.3
-rw-r--r--@ 1 staff   7.1K Apr 23 17:16 tensor_tutorial.ipynb
-rw-r--r--@ 1 staff   7.1K Apr 23 17:16 tensor_tutorial.ipynb.1
-rw-r--r--@ 1 staff   7.1K Apr 23 17:16 tensor_tutorial.ipynb.2
-rw-r--r--@ 1 staff   7.1K Apr 23 17:16 tensor_tutorial.ipynb.3


--2017-05-06 18:02:57--  http://pytorch.org/tutorials/_downloads/tensor_tutorial.ipynb
Resolving pytorch.org... 192.30.252.153
Connecting to pytorch.org|192.30.252.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7315 (7.1K) [application/octet-stream]
Saving to: ‘tensor_tutorial.ipynb.3’

     0K .......                                               100% 2.42M=0.003s

2017-05-06 18:02:58 (2.42 MB/s) - ‘tensor_tutorial.ipynb.3’ saved [7315/7315]

--2017-05-06 18:02:58--  http://pytorch.org/tutorials/_downloads/autograd_tutorial.ipynb
Resolving pytorch.org... 192.30.252.153
Connecting to pytorch.org|192.30.252.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6449 (6.3K) [application/octet-stream]
Saving to: ‘autograd_tutorial.ipynb.3’

     0K ......                                                100% 61.5M=0s

2017-05-06 18:02:58 (61.5 MB/s) - ‘autograd_tutorial.ipynb.3’ saved [6449/6449]



In [1]:
import torch
x = torch.Tensor(5, 3)

print('x: {}'.format(x))

x: 
 0.0000e+00 -1.5846e+29  5.4353e-30
-1.0845e-19  6.1630e-33  1.4013e-45
 4.1412e-33  1.4013e-45  8.0392e-33
 1.4013e-45  5.8693e-33  1.4013e-45
 5.4054e-33  1.4013e-45  5.3207e-33
[torch.FloatTensor of size 5x3]



In [2]:
x = torch.rand(5, 3)
print(x)
print(x.size())


 0.5246  0.8948  0.8550
 0.0660  0.7591  0.6264
 0.9002  0.4140  0.1190
 0.6253  0.7102  0.8339
 0.7086  0.9752  0.2164
[torch.FloatTensor of size 5x3]

torch.Size([5, 3])


In [3]:
y = torch.rand(5, 3)
print(x + y)


 1.2505  1.2449  0.8633
 0.4939  1.4918  1.5744
 1.2365  1.0999  0.8417
 1.1553  1.1960  1.3604
 1.4752  1.9528  0.4506
[torch.FloatTensor of size 5x3]



In [4]:
print(torch.add(x, y))


 1.2505  1.2449  0.8633
 0.4939  1.4918  1.5744
 1.2365  1.0999  0.8417
 1.1553  1.1960  1.3604
 1.4752  1.9528  0.4506
[torch.FloatTensor of size 5x3]



In [5]:
result = torch.Tensor(5, 3)
torch.add(x, y, out=result)
print(result)
print(y.add_(x))


 1.2505  1.2449  0.8633
 0.4939  1.4918  1.5744
 1.2365  1.0999  0.8417
 1.1553  1.1960  1.3604
 1.4752  1.9528  0.4506
[torch.FloatTensor of size 5x3]


 1.2505  1.2449  0.8633
 0.4939  1.4918  1.5744
 1.2365  1.0999  0.8417
 1.1553  1.1960  1.3604
 1.4752  1.9528  0.4506
[torch.FloatTensor of size 5x3]



In [6]:
print(x[:, 1])


 0.8948
 0.7591
 0.4140
 0.7102
 0.9752
[torch.FloatTensor of size 5]



In [7]:
a = torch.ones(5)
print(a)
print(type(a))
print(a.numpy())
print(type(a.numpy()))


 1
 1
 1
 1
 1
[torch.FloatTensor of size 5]

<class 'torch.FloatTensor'>
[ 1.  1.  1.  1.  1.]
<class 'numpy.ndarray'>


In [9]:
import numpy as np
a = np.ones(5)
print(type(a))
print(type(torch.from_numpy(a)))
print(torch.from_numpy(a))

<class 'numpy.ndarray'>
<class 'torch.DoubleTensor'>

 1
 1
 1
 1
 1
[torch.DoubleTensor of size 5]



---
# 自動微分
多くのフレームワークでは変数のインスタンスが、入力となるデータ（直接変更を加えることが多いデータ）の他に、
その変数の勾配とどのクラスで作成されたかを保持することが多いです。
中には、学習時以外で勾配（ロス）を計算する無駄を省くために、学習中かどうかのフラグを持つこともあります。
PyTorchでは変数は`Variable`クラスを用います。

In [16]:
from torch.autograd import Variable

In [17]:
x = Variable(torch.ones(2, 2), requires_grad=True)
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



In [18]:
y = x + 2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



In [19]:
print(y.creator)

In [20]:
z = y * y * 3
out = z.mean()
print('z: {}, out: {}'.format(z, out))

z: Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
, out: Variable containing:
 27
[torch.FloatTensor of size 1]



ここでは自動微分が実際に動作していることを確認するために、強引に誤差逆伝播を行います。
多くのフレームワークで、モデルの出力や、誤差を表す変数で`variable.backward()`を実行することで勾配が伝播しますが、
この段階ではまだパラメータの更新は行われません。

In [21]:
out.backward()
print(x.grad)

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]

